In [6]:
import os 
import numpy as np
import cv2
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [7]:
tf.test.gpu_device_name()

''

In [8]:
i=int(1) 


In [9]:
data_dir = './'

categories = ['daisy','dandelion','rose','sunflower','tulip']
data = []

def make_data():
  for category in categories:
    global i 
    print(i)
    i=i+1;
    path = os.path.join(data_dir,category)
    label = categories.index(category)
    for img_name in os.listdir(path):
      image_path = os.path.join(path,img_name)
      image = cv2.imread(image_path)
      try:
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image = cv2.resize(image,(224,224))
        image = np.array(image,dtype=np.float32)
        data.append([image,label])
      except Exception as e:
        pass





make_data()

1
2
3
4
5


In [10]:
def load_data():
  np.random.shuffle(data)
  feature = []
  labels = []
  for img,label in data:
    feature.append(img)
    labels.append(label)
  feature = np.array(feature,dtype=np.float32)
  labels = np.array(labels)
  feature = feature/255.0
  return [feature,labels]

In [11]:
(features,labels) = load_data()
X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=0.1)
categories = ['daisy','dandelion','rose','sunflower','tulip']

In [12]:
X_train.shape

(3890, 224, 224, 3)

In [13]:
input_layer = tf.keras.layers.Input([224,224,3])
conv1 = tf.keras.layers.Conv2D(filters = 32,kernel_size=(5,5),padding='Same',activation='relu')(input_layer)
pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(conv1)
conv2 = tf.keras.layers.Conv2D(filters = 64,kernel_size=(3,3),padding='Same',activation='relu')(pool1)
pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(conv2)
conv3 = tf.keras.layers.Conv2D(filters = 96,kernel_size=(3,3),padding='Same',activation='relu')(pool2)
pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(conv3)
conv4 = tf.keras.layers.Conv2D(filters = 96,kernel_size=(3,3),padding='Same',activation='relu')(pool3)
pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(conv4)
flt1 = tf.keras.layers.Flatten()(pool4)
dn1 = tf.keras.layers.Dense(512,activation='relu')(flt1)
out = tf.keras.layers.Dense(5,activation='softmax')(dn1)

model = tf.keras.Model(input_layer,out)


In [14]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [15]:
model.fit(X_train,y_train,batch_size=100,epochs=10)

Epoch 1/10
 6/39 [===>..........................] - ETA: 7:28 - loss: 1.8207 - accuracy: 0.2085

ResourceExhaustedError:  OOM when allocating tensor with shape[100,224,224,32] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node gradient_tape/model/max_pooling2d/MaxPool/MaxPoolGrad (defined at <ipython-input-15-d95fada4a007>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_902]

Function call stack:
train_function


In [ ]:
model.evaluate(X_test,y_test,verbose=1)

In [ ]:
prediction = model.predict(X_test)

plt.figure(figsize=(9,9))

for i in range(9):
  plt.subplot(3,3,i+1)
  plt.imshow(X_test[i])
  
  plt.xlabel('actual : '+categories[y_test[i]]+'\n'+'Predicted: \n'+
             categories[np.argmax(prediction[i])])
  print('\n')
plt.xticks([])

plt.show()
